In [1]:
import pandas as pd
from konlpy.tag import Twitter

twitter = Twitter()

/anaconda3/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
#300개의 텍스트를 수집한다. 명사만 추출하여 TDM을 만든다.
dataset = pd.read_csv("/Users/SAEROM/reple.csv")
#첫 5행을 불러옴
dataset.head() 

,reple,label
0,아니 나도 톰홀랜드 ㅈㄴ 좋아해서 연애설난거 슬프긴한데 왜 얼평하고 아깝다하고 안어...,1
1,눈이 낮구나래 씨발 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 말하는 뽄새 역겹네,1
2,쓰니는 톰홀랜드 덕질 별로 하지도 않았을것같다 ㅋㅋ말하는 꼬라지가,1
3,일단 함소원이 진화 ATM 으로 길들이려는거부터 진화 빡쳐서 도망갈 필임. 미친 한...,1
4,아니 솔직히 진화같은 애가 왜 저런 성괴 아줌마 만났는지 노이해...잘생기고 어리고...,1


In [3]:
data_X = dataset["reple"]

In [4]:
#2글자 이상인 명사만 추출하는 함수를 만든다.
def kor_noun(text):
    words = []
    for w in twitter.nouns(text):
        if len(w) > 1:
            words.append(w)
    return words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
cv = CountVectorizer(tokenizer=kor_noun, max_features=1000)

In [7]:
#댓글을 파일에서 읽어들일 때 자동으로 숫자로 변환되는 경우, 강제 문자열로 변환해준다.
reples = dataset['reple'].astype(str)

In [8]:
tdm = cv.fit_transform(reples)
words = cv.get_feature_names()

In [9]:
sorted(zip(tdm.sum(axis=0).flat, words), reverse=True)[:20]

[(64, '새끼'),
 (37, '남자'),
 (36, '진짜'),
 (32, '여자'),
 (27, '사람'),
 (21, '생각'),
 (17, '씨발'),
 (16, '한남'),
 (13, '정도'),
 (12, '그냥'),
 (11, '일본'),
 (11, '문제'),
 (10, '친구'),
 (10, '존나'),
 (10, '얼굴'),
 (10, '공부'),
 (9, '연애'),
 (8, '정말'),
 (8, '자기'),
 (8, '인생')]

In [10]:
#Topic Model 돌리기
#gensim에서는 고유한 데이터 형식을 쓰므로 CountVectorizer로 만든 tdm을 변환해주어야 함.
from gensim.matutils import Sparse2Corpus
corpus = Sparse2Corpus(tdm.T)

In [74]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(corpus = corpus, id2word=dict(enumerate(words)),
              num_topics=10, passes=10, iterations=50, random_state=1234)

In [52]:
lda.show_topic(0)

[('생각', 0.018201942),
 ('정말', 0.015763307),
 ('한남', 0.013216359),
 ('여자', 0.013208857),
 ('사람', 0.013201531),
 ('남자', 0.010675383),
 ('원래', 0.010575335),
 ('이건', 0.008076573),
 ('시험', 0.008073284),
 ('사업', 0.008072711)]

In [53]:
lda.show_topic(1)

[('새끼', 0.028867133),
 ('씨발', 0.026855497),
 ('진짜', 0.022658825),
 ('병신', 0.011490192),
 ('여자', 0.011478924),
 ('보고', 0.009254047),
 ('틀니', 0.009252455),
 ('정도', 0.009252225),
 ('그냥', 0.007625043),
 ('인생', 0.0074234772)]

In [54]:
lda.show_topic(2)

[('새끼', 0.031182652),
 ('여자', 0.01815018),
 ('진짜', 0.013673887),
 ('남자', 0.011447567),
 ('타령', 0.009221148),
 ('그냥', 0.009220654),
 ('쪽바리', 0.009201536),
 ('선배', 0.0069961883),
 ('일베', 0.006995679),
 ('페미', 0.006995375)]

In [55]:
lda.show_topic(3)

[('생각', 0.026207447),
 ('얼굴', 0.011873876),
 ('사람', 0.011873124),
 ('홀랜드', 0.009008319),
 ('연애', 0.009006475),
 ('짝남', 0.009006275),
 ('한남', 0.008974375),
 ('하루', 0.006142728),
 ('친구', 0.0061425436),
 ('사진', 0.006142208)]

In [56]:
lda.show_topic(4)

[('사람', 0.038454946),
 ('남자', 0.01939092),
 ('세상', 0.009220876),
 ('한남', 0.0085648075),
 ('그거', 0.00856256),
 ('심리', 0.008559127),
 ('그냥', 0.008555501),
 ('문제', 0.0059619485),
 ('한국', 0.0058394354),
 ('막말', 0.0058366414)]

In [57]:
lda.show_topic(5)

[('새끼', 0.03604946),
 ('문제', 0.014085681),
 ('남자', 0.008627874),
 ('진짜', 0.008607688),
 ('이해', 0.0085932),
 ('종특', 0.008572514),
 ('저런', 0.005874987),
 ('머리', 0.0058742515),
 ('개새끼', 0.0058731968),
 ('나라', 0.005873141)]

In [68]:
reples[80]

'ㄱㅊ아 애들아 일본 곧 멀지않아 육지가 가라앉아서 없어질거임 ㅋㅋㅋㅋㅋㅋㅋㅅㅂ...진짜 ㅈ같네 일본새끼들'

In [69]:
#tdm에서 i번째 행을 gensim 형식으로 바꿔주는 함수를 만들자.
def tdm2doc(tdm, i):
    doc = []
    for i,n in enumerate(tdm[i].toarray().flat):
        if n>0:
            doc.append((i,n))
    return doc

In [70]:
doc = tdm2doc(tdm, 80)

In [71]:
#단어 번호와 단어의 사용횟수의 짝으로 표현됨
doc

[(406, 1), (634, 1), (681, 1), (824, 1)]

In [75]:
#50번 문서의 주제비율 확인
lda.get_document_topics(doc)

[(0, 0.020001352),
 (1, 0.8199786),
 (2, 0.02000203),
 (3, 0.020000704),
 (4, 0.020000735),
 (5, 0.020004032),
 (6, 0.020008547),
 (7, 0.020002939),
 (8, 0.020000331),
 (9, 0.02000076)]

In [76]:
#주요 토픽을 한 번에 보려면..
for t in range(10):
    topic =[word for word,p in lda.show_topic(t)]
    print(t, ' '.join(topic))

0 생각 진짜 사람 정말 이건 사업 한남 쓰레기 여자 자기
1 진짜 새끼 일본 대학 정도 저런 누가 관심 원숭이 지금
2 새끼 여자 보지 남자 선배 실존 사람 자꾸 처음 거리
3 생각 피부 친구 연락 주변 참석 친절 문제 여자 진짜
4 사람 남자 진화 느낌 새끼 아예 신경 한국 누가 막말
5 새끼 문제 친구 진짜 개새끼 머리 나라 오히려 대화 불쌍
6 새끼 진짜 남자 공부 존나 일본 씨발 여자 병신 나이
7 새끼 남자 타령 홀랜드 얼평 짝남 외모 세상 공부 일베
8 한남 여자 남자 사람 정말 시험 거지 종특 조카 가슴
9 씨발 외모 그냥 마취 남자 얼굴 사람 여자 연애 성격


In [78]:
for i in lda.show_topic(7):
    print(i)

('새끼', 0.03646199)
('남자', 0.014798865)
('타령', 0.01119128)
('홀랜드', 0.011191277)
('얼평', 0.011191218)
('짝남', 0.011191186)
('외모', 0.007627576)
('세상', 0.007613273)
('공부', 0.0075811567)
('일베', 0.007581137)


In [90]:
from bs4 import BeautifulSoup
import requests

In [94]:
def find_dic(topic): #lda.show_topic
    li = []
    for t in range(10):
        topics = [word for word, p in lda.show_topic(t)]
        
        for topic in topics:
            url = "https://stdict.korean.go.kr/" + "api/search.do?certkey_no=887&key=DBB8DDBF7FF1B68CD5B631FDD8EB16FF" + "&type_search=search&q=" + topic;
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            items = soup.find_all('item')
            count = 0
            
            for item in items: count+= 1
                
            if(count==0):
                li.append(topic)
    return li
   

In [95]:
find_dic(lda.show_topic)

['존나', '씨발', '홀랜드', '얼평', '짝남', '일베', '종특', '씨발']